In [ ]:
!pip install accelerate==0.21.0 bitsandbytes==0.41.0 datasets==2.14.2 einops==0.6.1 gradio==3.37.0 peft==0.4.0 protobuf==4.23.4 scipy==1.11.1 sentencepiece==0.1.99 transformers==4.31.0 trl==0.5.0> /dev/null 2>&1

#### Listing 1

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig, TextIteratorStreamer
from trl.trainer.utils import PeftSavingCallback
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model, PeftModel
from peft.tuners.lora import LoraLayer
from datasets import load_dataset
from threading import Thread
import gradio as gr
import transformers
import torch

model_id = "openlm-research/open_llama_7b_v2"
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="right", use_fast=False)

prompt = "Hello World!"
inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt")['input_ids']
print(inputs)
# tensor([[ 8479,  2695, 29578]])

#### Listing 2

In [ ]:
output = tokenizer.decode(inputs.squeeze())
print(output)
# Hello World!

#### Listing 3

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

#### Listing 4

In [ ]:
model_id = "thisserand/open_llama_7b_v2_sharded"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="right", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             load_in_4bit=True, # Lade Modell 4-Bit quantisiert
                                             torch_dtype=torch.bfloat16, # Verwende BFloat16-Datentyp für Berechnungen
                                             device_map="auto" # Weise Modellgewichtungen automatisch zu: GPU > CPU > Festplatte
                                             )

#### Listing 5

In [ ]:
torch.manual_seed(1337)

generation_config = GenerationConfig(
    temperature=0.1, # Diversität des generierten Texts (<1 sorgt für eher deterministische Ergebnisse, während >1 die Kreativität und Zufälligkeit erhöht)
    top_k=40, # Wähle den nächsten Token aus den 40 wahrscheinlichsten nächsten Token
    top_p=0.75, # Wähle den nächsten Ausgabe-Token aus einer Teilmenge aller wahrscheinlichsten nächsten Token aus, wobei die kumulative Wahrscheinlichkeit der Teilmenge größer als 0.75 ist. Da top_k ebenfalls definiert ist enthält die Teilmenge maximal 40 Token.
)

output_tokens = model.generate(
    input_ids=inputs.to(model.device),
    generation_config=generation_config,
    max_new_tokens=100, # Anzahl der maximal zu generierenden Token
)

output_text = tokenizer.decode(output_tokens.squeeze())
print(output_text)

#### Listing 6

In [ ]:
def generate_text(prompt, model, tokenizer, generation_config):
    inputs = tokenizer(prompt, return_tensors="pt")['input_ids']
    output_tokens = model.generate(
        input_ids=inputs.to(model.device),
        generation_config=generation_config,
        max_new_tokens=100,
    )
    output_text = tokenizer.decode(output_tokens.squeeze(), skip_special_tokens=True)
    return output_text

prompt = "Frage: Wie viele Bundesländer hat Deutschland? Antwort:"
generated_text = generate_text(prompt, model, tokenizer, generation_config)
print(generated_text)

#### Listing 7

In [ ]:
from datasets import load_dataset

dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")
print(dataset[5]['text'])

#### Listing 8

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # Modell wird 4-Bit quantisiert geladen
    bnb_4bit_use_double_quant=True, # Quantisiuerungskonstanten werden ebenfalls quantisiert (reduziert Speicherbedarf)
    bnb_4bit_quant_type="nf4", # spezifischer Datentyp 'nf4' wird als 4-Bit Datentyp verwendet (optimal für normalverteilte Modellgewichtungen)
    bnb_4bit_compute_dtype=torch.bfloat16 # Datentyp 'bfloat16' wird für Berechnungen verwendet. Hierfür werden 4-Bit Modellgewichtungen bzw. Teile der Gewichtungen zur Laufzeit zu BFloat16 dequantisiert sodass Matrizenmultiplikation mit 16-bit Genauigkeit durchgeführt werden können
)

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.bfloat16)
model.config.torch_dtype = torch.bfloat16

# Modell für QLoRA Fine-Tuning vorbereiten
model = prepare_model_for_kbit_training(model)

# Low Rank Adapter (LoRA) konfigurieren
lora_config = LoraConfig(
    r=64, # Rang der Matrixfaktorisierungen
    lora_alpha=16, # LoRA Skalierungsfaktor
    target_modules=['_proj'], # Schichten in denen Adapter hinzugefügt werden
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Adapter zum Modell hinzufügen
model = get_peft_model(model, lora_config)

# Hinzugefügte Adapter zu Datentyp BFloat16 konvertieren
for name, module in model.named_modules():
    if isinstance(module, LoraLayer):
        module = module.to(torch.bfloat16)
    if 'norm' in name:
        # Normierung benötigt wenige Parameter, höhere Genauigkeit der Normierung sorgt für stabileres Fine-Tuning
        module = module.to(torch.float32)
    if 'lm_head' in name or 'embed_tokens' in name:
        if hasattr(module, 'weight'):
            if module.weight.dtype == torch.float32:
                module = module.to(torch.bfloat16)

#### Listing 9

In [ ]:
MAX_SEQUENCE_LENGTH = 512

def preprocess_function(sample):
    preprocessed_prompt = ""

    # Jeder Turn besteht aus menschlicher Anweisung und Antwort vom Assistenten
    turn_delimiter = "### Human:"
    turns = [turn_delimiter + turn for turn in sample['text'].split(turn_delimiter) if turn]
    for turn in turns:
        # Verarbeite Turn nur wenn Antwort vom Assistenten ebenfalls enthalten ist
        if "### Assistant:" in turn:
            # Jeder Turn beginnt mit einem Satzbeginn- und endet mit einem Satzende-Token
            preprocessed_prompt += tokenizer.bos_token + turn + tokenizer.eos_token
    result = tokenizer(preprocessed_prompt, max_length=MAX_SEQUENCE_LENGTH, truncation=True, add_special_tokens=False)

    result["labels"] = result["input_ids"].copy()
    return result

# Datensatz vorverarbeiten
dataset = dataset.map(preprocess_function, remove_columns=["text"])

# pad_token ist standardmäßig nicht definiert, wird für das Fine-Tuning jedoch benötigt
tokenizer.pad_token = 0

#### Listing 10

In [ ]:
training_arguments = transformers.TrainingArguments(
        per_device_train_batch_size=1, # Batchgröße je GPU
        gradient_accumulation_steps=16, # Gradienten werden über zwei Batches (2*16=32) akkumuliert und Low-Rank-Adapter werden erst dann aktualisiert (nicht per Batch); Methode um Speicher-effizient die Batchgröße zu vergößern
        max_steps=1875, # maximale Anzahl an Trainingsiterationen (pro Iteration werden Modellgewichtungen optimiert)
        learning_rate=2e-4, # Lernrate (2e-4 liefert gute Ergebnisse für 7B-LLMs)
        warmup_ratio=0.03, # Anzahl Iterationen in denen tatsächliche Lernrate linear von 0 zu definierter Lernrate erhöht wird (stabilisiert Trainingsprozess)
        lr_scheduler_type = "constant", # Nachdem Warmup-Phase abgeschlossen ist bleibt die Lernrate konstant
        bf16=True, # sorgt für 16-Bit (anstatt 32-Bit) Fine-Tuning, betrifft zu optimierende Low-Rank Adapter sowie die zur Optimierung benötigten Gradienten
        logging_steps=10, # Interval in dem Trainingsfortschritt geloggt wird
        output_dir="outputs", # Ausgabeordner in dem die Low Rank Adapter gespeichert werden
        optim="paged_adamw_32bit", # Zu verwendender Optimizer (aktualisiert/optimiert Modellgewichtungen während des Fine-Tunings)
        gradient_checkpointing=True, # Verringert Speicherbedarf und vermeidet OOM-Errors, macht das Training jedoch langsamer
        max_grad_norm = 0.3, # Vermeidet das Gradienten während des Trainings zu groß werden (dient der Stabilisierung des Trainings)
)

trainer = transformers.Trainer(
            model=model,
            args=training_arguments,
            # Der data_collator fügt dynamisch Padding-Token hinzu, sodass alle Sequenzen die gleiche Länge haben
            data_collator=transformers.DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt"),
            train_dataset=dataset,
            tokenizer=tokenizer,
            # Das PeftSavingCallback sorgt dafür dass ausschließlich die Low Rank Adapter abgespeichert werden
            callbacks=[PeftSavingCallback]
        )

# Deaktivieren des Caches verringert Speicherverbrauch. Sollte bei Inferenz jedoch wieder aktiviert werden, da es für schnellere Textgenerierung sorgt
model.config.use_cache = False

trainer.train()
# Speichern der gefinetuneten Low Rank Adapter
trainer.save_model()

#### Listing 11

In [ ]:
def load_base_model(model_id):
    return AutoModelForCausalLM.from_pretrained(
        model_id,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type='nf4'
        )
    )

adapters_id = "outputs"
base_model = load_base_model(model_id)
model = PeftModel.from_pretrained(base_model, adapters_id)
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="right", use_fast=False)

#### Listing 12

In [ ]:
def get_formatted_prompt(prompt):
    return f"### Human: {prompt} ### Assistant:"

prompt = "Schreibe den Bubble Sort Algorithmus in Python."
formatted_prompt = get_formatted_prompt(prompt)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(inputs=inputs.input_ids, max_new_tokens=300)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

#### Listing 13

In [ ]:
def stream_text_generation(model, tokenizer, prompt, generation_config):
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, Timeout=5, skip_special_tokens=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    kwargs = dict(input_ids=inputs["input_ids"], generation_config=generation_config,
                  streamer=streamer, max_new_tokens=512)
    thread = Thread(target=model.generate, kwargs=kwargs)
    thread.start()
    return streamer


generated_text = ""
text_generation_stream = stream_text_generation(model, tokenizer, formatted_prompt, generation_config)
for decoded_token in text_generation_stream:
    generated_text += decoded_token
    print(generated_text)

#### Listing 14

In [ ]:
with gr.Blocks() as llm_ui:
  chatbot = gr.Chatbot()
  prompt_textbox = gr.Textbox()
  clear_button = gr.Button("Clear")

  def format_history_to_prompt(history):
    prompt = ""
    for i, turn in enumerate(history):
      human_instruction, assistant_completion = turn
      # Da der Tokenizer automatisch den Satzbeginn-Token voranstellt, wird dieser der ersten Anweisung nicht vorangestellt
      if i > 0:
        prompt += tokenizer.bos_token
      # füge '### Human: {human_instruction} ### Assistant:' hinzu
      prompt += get_formatted_prompt(human_instruction)
      # falls vorhanden, ergänze '{assistant_completion}</s>'
      if assistant_completion is not None:
        prompt += assistant_completion + tokenizer.eos_token
    return prompt

  def user(user_message, history):
      return "", history + [[user_message, None]]

  def bot(history):
      prompt = format_history_to_prompt(history)

      history[-1][1] = ""
      text_generation_stream = stream_text_generation(model, tokenizer, prompt, generation_config)
      for decoded_token in text_generation_stream:
          history[-1][1] += decoded_token
          yield history

  prompt_textbox.submit(user, [prompt_textbox, chatbot], [prompt_textbox, chatbot], queue=False).then(bot, chatbot, chatbot)
  clear_button.click(lambda: None, None, chatbot, queue=False)
  llm_ui.queue()
  llm_ui.launch()

#### Listing 15

In [ ]:
prompt = "Ich habe extreme Schmerzen im unteren Rücken."
formatted_prompt = get_formatted_prompt(prompt)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(inputs=inputs.input_ids, max_new_tokens=300)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

#### Listing 16

In [ ]:
base_model = load_base_model(model_id)
base_model = prepare_model_for_kbit_training(base_model)
# Durch Hinzufügen des Parameters is_trainable=True werden die
# Low Rank Adapter so hinzugefügt, dass sie beim Fine-Tuning
# optimiert werden können
model = PeftModel.from_pretrained(base_model, adapters_id, is_trainable=True)

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="right", use_fast=False)
tokenizer.pad_token = 0

# Datensatz vorverarbeiten
dataset = load_dataset("thisserand/health_care_german", split="train")
dataset = dataset.map(preprocess_function, remove_columns=["text"])

training_arguments.max_steps = 200
training_arguments.output_dir = "outputs_health_care"

trainer = transformers.Trainer(
            model=model,
            args=training_arguments,
            data_collator=transformers.DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt"),
            train_dataset=dataset,
            tokenizer=tokenizer,
            callbacks=[PeftSavingCallback]
        )

# Deaktivieren des Caches verringert Speicherverbrauch. Sollte bei Inferenz jedoch wieder aktiviert werden, da es für schnellere Textgenerierung sorgt
model.config.use_cache = False

trainer.train()
# Speichern der gefinetuneten Low Rank Adapter
trainer.save_model()

#### Listing 17

In [ ]:
adapters_id = "outputs_health_care"
base_model = load_base_model(model_id)
model = PeftModel.from_pretrained(base_model, adapters_id)
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="right", use_fast=False)

prompt = "Ich habe extreme Schmerzen im unteren Rücken."
formatted_prompt = get_formatted_prompt(prompt)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(inputs=inputs.input_ids, max_new_tokens=300)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))